In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from sklearn.preprocessing import LabelEncoder
import nltk
from sklearn.model_selection import train_test_split
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

import time

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/gdrive


In [2]:
df_data = pd.read_csv('gdrive/My Drive/TA/Copy of DoS.csv')
df_data.head()

,frame.time_delta,frame.time_delta_displayed,frame.time_epoch,frame.time_invalid,frame.time_relative,ip.src,ip.dst,tcp.srcport,tcp.dstport,eth.src,eth.dst,frame.cap_len,frame.coloring_rule.name,frame.coloring_rule.string,frame.comment,frame.comment.expert,frame.encap_type,frame.file_off,frame.ignored,frame.incomplete,frame.interface_id,frame.interface_name,frame.len,frame.link_nr,frame.marked,frame.md5_hash,frame.number,frame.offset_shift,mqtt.clientid,mqtt.clientid_len,mqtt.conack.flags,mqtt.conack.flags.reserved,mqtt.conack.flags.sp,mqtt.conack.val,mqtt.conflag.cleansess,mqtt.conflag.passwd,mqtt.conflag.qos,mqtt.conflag.reserved,mqtt.conflag.retain,mqtt.conflag.uname,mqtt.conflag.willflag,mqtt.conflags,mqtt.dupflag,mqtt.hdrflags,mqtt.kalive,mqtt.len,mqtt.msg,mqtt.msgid,mqtt.msgtype,mqtt.passwd,mqtt.passwd_len,mqtt.proto_len,mqtt.protoname,mqtt.qos,mqtt.retain,mqtt.sub.qos,mqtt.suback.qos,mqtt.topic,mqtt.topic_len,mqtt.username,mqtt.username_len,mqtt.ver,mqtt.willmsg,mqtt.willmsg_len,mqtt.willtopic,mqtt.willtopic_len,type
0,0.000000,0.000000,1.522235e+09,NaN,0.000000,192.168.1.1,192.168.1.196,22.0,51749.0,18:a6:f7:eb:77:26,30:5a:3a:62:72:80,122,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,122,NaN,0,NaN,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal
1,0.001922,0.001922,1.522235e+09,NaN,0.001922,192.168.1.1,192.168.1.196,22.0,51749.0,18:a6:f7:eb:77:26,30:5a:3a:62:72:80,190,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,190,NaN,0,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal
2,0.001010,0.001010,1.522235e+09,NaN,0.002932,192.168.1.196,192.168.1.1,51749.0,22.0,30:5a:3a:62:72:80,18:a6:f7:eb:77:26,54,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,54,NaN,0,NaN,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal
3,0.008597,0.008597,1.522235e+09,NaN,0.011529,192.168.1.1,192.168.1.196,22.0,51749.0,18:a6:f7:eb:77:26,30:5a:3a:62:72:80,186,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,186,NaN,0,NaN,4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal
4,0.002925,0.002925,1.522235e+09,NaN,0.014454,192.168.1.196,192.168.1.1,51749.0,22.0,30:5a:3a:62:72:80,18:a6:f7:eb:77:26,54,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,54,NaN,0,NaN,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal


# Sebelum Preprocessing

In [3]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94625 entries, 0 to 94624
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   frame.time_delta            94625 non-null  float64
 1   frame.time_delta_displayed  94625 non-null  float64
 2   frame.time_epoch            94625 non-null  float64
 3   frame.time_invalid          0 non-null      float64
 4   frame.time_relative         94625 non-null  float64
 5   ip.src                      93592 non-null  object 
 6   ip.dst                      93592 non-null  object 
 7   tcp.srcport                 88460 non-null  float64
 8   tcp.dstport                 88460 non-null  float64
 9   eth.src                     94625 non-null  object 
 10  eth.dst                     94625 non-null  object 
 11  frame.cap_len               94625 non-null  int64  
 12  frame.coloring_rule.name    0 non-null      float64
 13  frame.coloring_rule.string  0 n

# Preprocessing

In [4]:
print (df_data.isnull().sum())

frame.time_delta                  0
frame.time_delta_displayed        0
frame.time_epoch                  0
frame.time_invalid            94625
frame.time_relative               0
                              ...  
mqtt.willmsg                  94625
mqtt.willmsg_len              94625
mqtt.willtopic                94625
mqtt.willtopic_len            94625
type                              0
Length: 67, dtype: int64


In [5]:
df_data.drop(['frame.time_invalid','frame.coloring_rule.name', 'frame.coloring_rule.string', 'frame.comment', 'frame.comment.expert', 'frame.file_off', 'frame.incomplete', 'frame.interface_id', 'frame.interface_name', 'frame.link_nr', 'frame.md5_hash', 'mqtt.passwd', 'mqtt.passwd_len', 'mqtt.username', 'mqtt.username_len', 'mqtt.willmsg', 'mqtt.willmsg_len', 'mqtt.willtopic', 'mqtt.willtopic_len'], axis=1, inplace = True)
print (df_data.isnull().sum())

frame.time_delta                  0
frame.time_delta_displayed        0
frame.time_epoch                  0
frame.time_relative               0
ip.src                         1033
ip.dst                         1033
tcp.srcport                    6165
tcp.dstport                    6165
eth.src                           0
eth.dst                           0
frame.cap_len                     0
frame.encap_type                  0
frame.ignored                     0
frame.len                         0
frame.marked                      0
frame.number                      0
frame.offset_shift                0
mqtt.clientid                 94302
mqtt.clientid_len             94302
mqtt.conack.flags             93855
mqtt.conack.flags.reserved    93855
mqtt.conack.flags.sp          93855
mqtt.conack.val               93855
mqtt.conflag.cleansess        94302
mqtt.conflag.passwd           94302
mqtt.conflag.qos              94302
mqtt.conflag.reserved         94302
mqtt.conflag.retain         

In [6]:
df_data.drop(['mqtt.clientid','mqtt.clientid_len', 'mqtt.conack.flags', 'mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conack.val', 'mqtt.conflag.cleansess', 'mqtt.conflag.passwd', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 'mqtt.conflag.retain', 'mqtt.conflag.uname', 'mqtt.conflag.willflag', 'mqtt.conflags', 'mqtt.kalive', 'mqtt.msgid', 'mqtt.proto_len', 'mqtt.protoname', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.ver'], axis=1, inplace = True)
print (df_data.isnull().sum())

frame.time_delta                  0
frame.time_delta_displayed        0
frame.time_epoch                  0
frame.time_relative               0
ip.src                         1033
ip.dst                         1033
tcp.srcport                    6165
tcp.dstport                    6165
eth.src                           0
eth.dst                           0
frame.cap_len                     0
frame.encap_type                  0
frame.ignored                     0
frame.len                         0
frame.marked                      0
frame.number                      0
frame.offset_shift                0
mqtt.dupflag                  56161
mqtt.hdrflags                 55090
mqtt.len                      55090
mqtt.msg                      56892
mqtt.msgtype                  55090
mqtt.qos                      56161
mqtt.retain                   56161
mqtt.topic                    56888
mqtt.topic_len                56157
type                              0
dtype: int64


In [7]:
le = LabelEncoder()
 
df_data['type']= le.fit_transform(df_data['type'])
df_data['frame.encap_type']= le.fit_transform(df_data['frame.encap_type'])

print(df_data['type'])

0        1
1        1
2        1
3        1
4        1
        ..
94620    1
94621    1
94622    1
94623    1
94624    1
Name: type, Length: 94625, dtype: int64


In [8]:
df_data['ip.src'] = df_data['ip.src'].fillna(df_data['ip.src'].mode()[0])
df_data['ip.dst'] = df_data['ip.dst'].fillna(df_data['ip.dst'].mode()[0])
df_data['tcp.srcport'] = df_data['tcp.srcport'].fillna(df_data['tcp.srcport'].mean())
df_data['tcp.dstport'] = df_data['tcp.dstport'].fillna(df_data['tcp.dstport'].mean())
df_data['mqtt.dupflag'] = df_data['mqtt.dupflag'].fillna(0)
df_data['mqtt.hdrflags'] = df_data['mqtt.hdrflags'].fillna(0)
df_data['mqtt.len'] = df_data['mqtt.len'].fillna(0)
df_data['mqtt.msg'] = df_data['mqtt.msg'].fillna(0)
df_data['mqtt.msgtype'] = df_data['mqtt.msgtype'].fillna(0)
df_data['mqtt.qos'] = df_data['mqtt.qos'].fillna(0)
df_data['mqtt.retain'] = df_data['mqtt.retain'].fillna(0)
df_data['mqtt.topic'] = df_data['mqtt.topic'].fillna(0)
df_data['mqtt.topic_len'] = df_data['mqtt.topic_len'].fillna(0)

print(df_data.isna().sum())

frame.time_delta              0
frame.time_delta_displayed    0
frame.time_epoch              0
frame.time_relative           0
ip.src                        0
ip.dst                        0
tcp.srcport                   0
tcp.dstport                   0
eth.src                       0
eth.dst                       0
frame.cap_len                 0
frame.encap_type              0
frame.ignored                 0
frame.len                     0
frame.marked                  0
frame.number                  0
frame.offset_shift            0
mqtt.dupflag                  0
mqtt.hdrflags                 0
mqtt.len                      0
mqtt.msg                      0
mqtt.msgtype                  0
mqtt.qos                      0
mqtt.retain                   0
mqtt.topic                    0
mqtt.topic_len                0
type                          0
dtype: int64


In [9]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94625 entries, 0 to 94624
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   frame.time_delta            94625 non-null  float64
 1   frame.time_delta_displayed  94625 non-null  float64
 2   frame.time_epoch            94625 non-null  float64
 3   frame.time_relative         94625 non-null  float64
 4   ip.src                      94625 non-null  object 
 5   ip.dst                      94625 non-null  object 
 6   tcp.srcport                 94625 non-null  float64
 7   tcp.dstport                 94625 non-null  float64
 8   eth.src                     94625 non-null  object 
 9   eth.dst                     94625 non-null  object 
 10  frame.cap_len               94625 non-null  int64  
 11  frame.encap_type            94625 non-null  int64  
 12  frame.ignored               94625 non-null  int64  
 13  frame.len                   946

In [10]:
df_data['ip.src'] = df_data['ip.src'].apply(pd.to_numeric, errors='coerce')
df_data['ip.dst'] = df_data['ip.dst'].apply(pd.to_numeric, errors='coerce')
df_data['eth.src'] = df_data['eth.src'].apply(pd.to_numeric, errors='coerce')
df_data['eth.dst'] = df_data['eth.dst'].apply(pd.to_numeric, errors='coerce')
df_data['mqtt.hdrflags'] = df_data['mqtt.hdrflags'].apply(pd.to_numeric, errors='coerce')
df_data['mqtt.msg'] = df_data['mqtt.msg'].apply(pd.to_numeric, errors='coerce')
df_data['mqtt.topic'] = df_data['mqtt.topic'].apply(pd.to_numeric, errors='coerce')

df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94625 entries, 0 to 94624
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   frame.time_delta            94625 non-null  float64
 1   frame.time_delta_displayed  94625 non-null  float64
 2   frame.time_epoch            94625 non-null  float64
 3   frame.time_relative         94625 non-null  float64
 4   ip.src                      0 non-null      float64
 5   ip.dst                      0 non-null      float64
 6   tcp.srcport                 94625 non-null  float64
 7   tcp.dstport                 94625 non-null  float64
 8   eth.src                     0 non-null      float64
 9   eth.dst                     0 non-null      float64
 10  frame.cap_len               94625 non-null  int64  
 11  frame.encap_type            94625 non-null  int64  
 12  frame.ignored               94625 non-null  int64  
 13  frame.len                   946

In [11]:
print(df_data.isna().sum())

frame.time_delta                  0
frame.time_delta_displayed        0
frame.time_epoch                  0
frame.time_relative               0
ip.src                        94625
ip.dst                        94625
tcp.srcport                       0
tcp.dstport                       0
eth.src                       94625
eth.dst                       94625
frame.cap_len                     0
frame.encap_type                  0
frame.ignored                     0
frame.len                         0
frame.marked                      0
frame.number                      0
frame.offset_shift                0
mqtt.dupflag                      0
mqtt.hdrflags                 39535
mqtt.len                          0
mqtt.msg                      37493
mqtt.msgtype                      0
mqtt.qos                          0
mqtt.retain                       0
mqtt.topic                    37737
mqtt.topic_len                    0
type                              0
dtype: int64


In [12]:
df_data.drop(['ip.src', 'ip.dst', 'eth.src', 'eth.dst'], axis=1, inplace = True)
print(df_data.isna().sum())

frame.time_delta                  0
frame.time_delta_displayed        0
frame.time_epoch                  0
frame.time_relative               0
tcp.srcport                       0
tcp.dstport                       0
frame.cap_len                     0
frame.encap_type                  0
frame.ignored                     0
frame.len                         0
frame.marked                      0
frame.number                      0
frame.offset_shift                0
mqtt.dupflag                      0
mqtt.hdrflags                 39535
mqtt.len                          0
mqtt.msg                      37493
mqtt.msgtype                      0
mqtt.qos                          0
mqtt.retain                       0
mqtt.topic                    37737
mqtt.topic_len                    0
type                              0
dtype: int64


In [13]:
df_data['mqtt.hdrflags'] = df_data['mqtt.hdrflags'].fillna(df_data['mqtt.hdrflags'].mean())
df_data['mqtt.msg'] = df_data['mqtt.msg'].fillna(df_data['mqtt.msg'].mean())
df_data['mqtt.topic'] = df_data['mqtt.topic'].fillna(df_data['mqtt.topic'].mean())

print(df_data.isna().sum())

frame.time_delta              0
frame.time_delta_displayed    0
frame.time_epoch              0
frame.time_relative           0
tcp.srcport                   0
tcp.dstport                   0
frame.cap_len                 0
frame.encap_type              0
frame.ignored                 0
frame.len                     0
frame.marked                  0
frame.number                  0
frame.offset_shift            0
mqtt.dupflag                  0
mqtt.hdrflags                 0
mqtt.len                      0
mqtt.msg                      0
mqtt.msgtype                  0
mqtt.qos                      0
mqtt.retain                   0
mqtt.topic                    0
mqtt.topic_len                0
type                          0
dtype: int64


# Setelah Preprocessing

In [14]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94625 entries, 0 to 94624
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   frame.time_delta            94625 non-null  float64
 1   frame.time_delta_displayed  94625 non-null  float64
 2   frame.time_epoch            94625 non-null  float64
 3   frame.time_relative         94625 non-null  float64
 4   tcp.srcport                 94625 non-null  float64
 5   tcp.dstport                 94625 non-null  float64
 6   frame.cap_len               94625 non-null  int64  
 7   frame.encap_type            94625 non-null  int64  
 8   frame.ignored               94625 non-null  int64  
 9   frame.len                   94625 non-null  int64  
 10  frame.marked                94625 non-null  int64  
 11  frame.number                94625 non-null  int64  
 12  frame.offset_shift          94625 non-null  float64
 13  mqtt.dupflag                946